# We’ve developed a streamlined AI-driven meeting analysis tool that effortlessly turns audio into insights. Using OpenAI’s Whisper‑1, our system transcribes meetings into accurate, time-stamped text from multilingual audio sources . That transcript is then run through Meta’s Llama 3.2 1B‑Instruct, a compact yet powerful instruction-tuned model capable of summarization, key action extraction, sentiment analysis, and conversational Q&A within a massive 128K token context 


# Imports 

In [6]:
import os
import torch
from openai import OpenAI
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
from IPython.display import Markdown, display

# Hugging Face & Openai Api Keys

In [2]:
HF_TOKEN = os.getenv("HF_TOKEN") 
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 

login(HF_TOKEN, add_to_git_credential=True)
openai = OpenAI(api_key=OPENAI_API_KEY)


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


# Models 

In [3]:
AUDIO_MODEL = "whisper-1"
LLAMA = "unsloth/Llama-3.2-1B-Instruct"  

audio_filename = r"C:\LLM_Projects\Project_4_\audio_files\Mushrooms_Zoom_Class_Meeting.mp3"


# Reading audio file 

In [8]:
with open(audio_filename, "rb") as audio_file:
    transcription = openai.audio.transcriptions.create(
        model=AUDIO_MODEL,
        file=audio_file,
        response_format="text"
    )
print(transcription)

uh he's talking about that mushroom's portability and importance with in regards to keeping aflame I think you know so joking about the hat although who knows the hat might burst into flames. It also just seemed like there were so many solutions so you just want us to write an essay on any one of them in depth right? Yeah absolutely because there's there's you know mycelium and and mushrooms and the way that he's using them and testing them and all the ways that he talked about there's dozens uh and so yeah well I mean it's again it's a solution paper in this class so that's only one page right and you've got to be you know as specific as possible so what stands out and thanks uh Riley for reminding me about the lecture recording lecture. I was already familiar with Paul's statement a little bit before watching this and I feel like while he something that stood out to me first was a lot of the scientific jargon sort of the parts that were a little harder to understand to me I think tha

# System and User Prompt 

In [12]:
system_message = (
    "You are an assistant that produces minutes of meetings from transcripts, "
    "with summary, key discussion ponts or any relevant information, "
)

user_prompt = f"""Below is an extract transcript of a  zoom class meeting. 
Please write minutes in markdown, including a summary.; 

{transcription}
"""

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
]


# Load Model and Tokenize Input 

In [13]:

tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token

inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cpu")

model = AutoModelForCausalLM.from_pretrained(
    LLAMA,
    torch_dtype=torch.float32
).to("cpu")


# Output 

In [14]:
streamer = TextStreamer(tokenizer)
outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)
response = tokenizer.decode(outputs[0])
display(Markdown(response))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 28 Jun 2025

You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion ponts or any relevant information,<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a  zoom class meeting. 
Please write minutes in markdown, including a summary.; 

uh he's talking about that mushroom's portability and importance with in regards to keeping aflame I think you know so joking about the hat although who knows the hat might burst into flames. It also just seemed like there were so many solutions so you just want us to write an essay on any one of them in depth right? Yeah absolutely because there's there's you know mycelium and and mushrooms and the way that he's using them and testing them and all the ways that he talked about there's dozens uh and so yeah well I mean it's again it's a solution paper in this class 

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 28 Jun 2025

You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion ponts or any relevant information,<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a  zoom class meeting. 
Please write minutes in markdown, including a summary.; 

uh he's talking about that mushroom's portability and importance with in regards to keeping aflame I think you know so joking about the hat although who knows the hat might burst into flames. It also just seemed like there were so many solutions so you just want us to write an essay on any one of them in depth right? Yeah absolutely because there's there's you know mycelium and and mushrooms and the way that he's using them and testing them and all the ways that he talked about there's dozens uh and so yeah well I mean it's again it's a solution paper in this class so that's only one page right and you've got to be you know as specific as possible so what stands out and thanks uh Riley for reminding me about the lecture recording lecture. I was already familiar with Paul's statement a little bit before watching this and I feel like while he something that stood out to me first was a lot of the scientific jargon sort of the parts that were a little harder to understand to me I think that um may have been attributed to one that he's really visibly very passionate about um the science and this new um these new developments on what mycelium can do and you know understanding even what exactly it is and two I feel like a lot of times his work is discredited because of um his connection to different people in the field and as well as um you know some of his more goofier stories that he's talked about on other platforms so I feel like sort of that scientific jargon or some of the ways that he got really into depth about it might be a way for him to try to explain to people that hey he is really serious about this work and there is a lot of scientific backing to it. Yeah excellent and he did address the notion of mycophobia. I mean if scientists are looking at this cave painting and on this cave painting it's clear that there's like dozens and dozens and dozens of mushrooms but no one's ever talked about it. I do think that he's right uh higher education has been mycophobic in ways um that's why actually for years he never talked about like psychedelic mushrooms um like none of the lectures for years and years and years and I think that is very intentional so that he doesn't like undermine the whole thing right because in reality he is very scientific and he is looking for very important statistical like measurability reliability statistical significance um like very very very very into the science of it right to the point where they're they're putting out tens of thousands of dollars to test these things and find out did we get something good against a virus did we get something important with whatever that is so yeah um somebody also put a ton of stigma around mycology meaning psychedelics yeah absolutely um so it's been refreshing I guess as I see him open up and talk about that and really talk about which he doesn't hear a lot he does not speak about the shamanistic properties the psychedelic properties the really the properties that aren't just sort of the science that we can grasp he alludes to them right uh this is the northwest coast this is a statistically shamanistically important people place for the height of people in this but he doesn't go into it too much because again I think forever he's tried to to not be that guy even though he's weird even though he's a genius even though he's him even though he's into mushrooms in like every way right because he points out his colleagues right and his colleagues are like he's like these are all very conservative people you know so you know that his background I mean he might have been young and excited about psychedelic mushrooms but the most important thing to him was you know the science of it with these very scientific people who were not not very interested uh you know as he explained in the in the other recreational you know kind of aspects it was more about taking mushrooms seriously which I guess if you're a mycologist and you work with that is a big part of the stigma around what you have to deal with you know good uh excellent what else stands out to you about this I think there are a lot of different things that stood out to me but one that I really noticed was the similarity between Paul and Mike Reynolds and how they have to like build their credibility and is so fragile so like we were just talking about I think like yes they are so passionate about it and yes they have great ideas but like their their appearance and their you know the way they talk about things like is so fragile in today's society so I just thought it was so interesting because they were they were very similar talking about their their passionate topics yeah almost as if you have to be like excessively boring to be understood or scientific which as we know some of the most brilliant people are are not the most boring um but yeah how do you also disseminate a message in there they know this and of course everybody around them is like how hard must it be to be involved want to do science with the environment and still have people be like could you just dress differently and you're like wait what are we still on square like that's like not even square one that's like square negative 31 um when we're trying to do serious work uh but that doesn't mean that these ideas always come from like typically serious people yeah um elon musk might be either good or really bad example of that and and sometimes it's brilliance is autistic sometimes it is uh like extrinsic and and and um oh it's just so many different approaches to genius or to people who really are thinking advanced thoughts um because right the michael reynolds piece we didn't touch on this when we're talking about it but remember when they asked him like well what else could you build a building out of that hasn't been thought of that was the question from serious people what else that hasn't been thought of can you think of like right there and he's like i don't know building a building out of freaking water i think his that's like basically his like answer there and so yeah that's why one of the things that i love about stamets is they're like well you can't do that with saltwater and he's like well and i'm sure it seems patronizing to the scientific community he's like well i'm an idiot let me try it and of course success right so i'm sure that uh you know a lot of people that think differently even if they're really smart boy uh people will do it just gives other people a chance to be able to tear that down easier though i don't understand the motivation of that that's why we study these things through a sociological lens right um always makes me think about that good contrast and good comparison between these two who else um i thought it was interesting that our environment has the loss of soil and how that's impacting our crops and our sustainability and that at least i've never heard of that fungi can be like transform wood and straw and grass and colonize it and just bring it into soil like turn it into soil i just thought that was pretty interesting and also pretty amazing because i didn't know about that yeah there's so many things just in this one video that are amazing when he talks about them eating rocks you know this is how they get their minerals this is how things break down so of course like the other piece it takes a long time right and i'm not sure because we do have a soil science person in there and i mentioned that before because somebody talked about that and so i'm going to speak about this in very general terms it takes a very long time to create a small amount of soil right like usable soil and i think that that's really really important when we talk about farming practices well why why might we not use pesticides to grow everything well if that's killing the biodiversity in the soil and renting the soil so that you can't grow things in it it's ultimately defeating the purpose right like how do we build more of that how do we make sure that that the things that are in the soil that allow it to break down can still survive you know amongst those conditions and in those ways so anyway yeah i thought that was very interesting too a lot of this stuff i don't even know how many i need a little clicker like in my hand the next time i watch this video because each time that my mind is like whoa whoa whoa what you know so many because there's gotta there's a lot of those moments in this piece that's a good one um that we don't often think of geez how long does it take to make soil that's quite valuable um or how is that made but yeah awesome all right who else i think it's interesting how it was really brief but he just randomly threw in like terraforming um and like casually intergalactic travel i thought that was just unexpected we were talking about earth and then all of a sudden he was just like oh by the way like these things can live in x y and z environments which could be helpful one day for you know stuff like space travel and stuff like that and people living in space so i thought that was a really cool random tangent and obviously you know not a solution to like here on earth but just a thing that he applied that to so i thought that was really interesting yeah i mean i don't know do these people get a lot of sleep you know because it's just like oh yeah what do you like<|eot_id|><|start_header_id|>assistant<|end_header_id|>

# Zoom Class Meeting Minutes

## Summary

The meeting discussed the presentation of Dr. Paul Stamets, a mycologist and expert in mycelium, on the importance of using mushrooms in various fields such as medicine, agriculture, and environmental sustainability. The discussion highlighted the scientific backing of his work, the stigma surrounding mycology, and the potential of mycelium to revolutionize different industries. The meeting also touched on the presentation's attempt to address the lack of scientific credibility and the importance of considering the scientific aspects of his work.

## Key Discussion Points

*   Dr. Stamets' presentation highlighted the importance of mycelium in various fields, including medicine, agriculture, and environmental sustainability.
*   The presentation addressed the stigma surrounding mycology, citing the lack of scientific credibility and the need for more research in the field.
*   Dr. Stamets discussed the potential of mycelium to revolutionize different industries, including the development of new medicines, sustainable agriculture practices, and space exploration.
*   The presentation also touched on the importance of considering the scientific aspects of his work, including the need for more research and the potential for breakthroughs in various fields.
*   The meeting highlighted the importance of addressing the stigma surrounding mycology and promoting a more scientific understanding of the field.

## Relevant Information

*   Mycelium: a network of fungal hyphae that can be used to produce a wide range of products, including medicines, food, and building materials.
*   Environmental Sustainability: the use of mycelium to promote sustainable agriculture practices and reduce the environmental impact of human activities.
*   Space Exploration: the use of mycelium to develop sustainable technologies for space exploration and colonization.

## Questions and Comments

*   Q: What is the significance of mycelium in various fields?
*   A: Mycelium has the potential to revolutionize different industries, including medicine, agriculture, and environmental sustainability.
*   Q: How can we promote a more scientific understanding of mycology?
*   A: We can promote a more scientific understanding of mycology by supporting research in the field, addressing the stigma surrounding mycology, and highlighting the potential of mycelium to solve complex problems.<|eot_id|>